## CS513 Data Cleansining

In [1]:
import pandas as pd
import numpy as np
from uszipcode import SearchEngine
# read the data
df = pd.read_excel('../Datasets/Food_Inspections_OpenRefine_cleaned.xlsx')
# display the dataframe head
df.head()

c:\Users\junyang\AppData\Local\Programs\Python\Python39\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
c:\Users\junyang\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
0,2079132.0,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517328.0,Restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616.0,2017-08-28,License,Pass,NaN,41.853651,-87.620534
1,2079129.0,JET'S PIZZA,JET'S PIZZA,2522268.0,Restaurant,Risk 2 (Medium),1025 W MADISON ST,CHICAGO,IL,60607.0,2017-08-28,License,Not Ready,NaN,41.881572,-87.653052
2,2079125.0,ROOM 1520,ROOM 1520,2446638.0,Special Event,Risk 3 (Low),301 N JUSTINE ST,CHICAGO,IL,60607.0,2017-08-28,License Re-inspection,Not Ready,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,41.886577,-87.665328
3,2079123.0,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517338.0,Restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616.0,2017-08-28,License,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.853651,-87.620534
4,2079105.0,CHARTWELLS,CICS WEST BELDEN CAMPUS,2549079.0,Charter School,Risk 1 (High),2245 N MCVICKER AVE,CHICAGO,IL,60639.0,2017-08-28,License Re-inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.921675,-87.776711


### count of null values in each column

In [2]:
print(df.isnull().sum())
# print(df.info())

Inspection ID          0
DBA Name               0
AKA Name            2487
License #              0
Facility Type       4508
Risk                   0
Address                1
City                  64
State                  8
Zip                    0
Inspection Date        0
Inspection Type        0
Results                0
Violations         30686
Latitude             533
Longitude            533
dtype: int64


In [3]:
# (df.isnull().sum() * 100 / df.index.size).round(2)
# print(df['City'].unique())
# df['License #'].apply(float.is_integer).all()
# df_num = df.select_dtypes(include=[float])

In [4]:
search = SearchEngine()

def getLocationByZipCode(zipcode):
    Location = search.by_zipcode(zipcode=zipcode)
    return Location
# print(getLocationByZipCode(60616))
# print(getLocationByZipCode(60616).lat)

#### find missing Latitude, Longitiude, City and State from dataset

In [5]:
missing_GPS = df.loc[pd.isnull(df['Latitude']) | pd.isnull(df['Longitude'])]
missing_city = df.loc[(df['City'].str.contains(r'[0-9]', na=False) | pd.isnull(df['City'])) & pd.notnull(df['Zip'])]
missing_state = df.loc[(df['State'].str.contains(r'[0-9]', na=False) | pd.isnull(df['State'])) & pd.notnull(df['Zip'])]
# print(missing_GPS.head().Latitude)
# print(missing_city.head().City)
# print(missing_state.head().State)

### fille the missing value by zip code SearchEngine

In [6]:
for row in missing_GPS.iterrows():
    index = row[0]
    zip = int(row[1].Zip)
    location = getLocationByZipCode(zip)
    df.iloc[index, df.columns.get_loc('Latitude')] = location.lat
    df.iloc[index, df.columns.get_loc('Longitude')] = location.lng
    #print(f'index: {index}  Zip: {zip} --> Latitude: {location.lat} Longitude: {location.lng}')

In [7]:
for row in missing_city.iterrows():
    index = row[0]
    zip = int(row[1].Zip)
    location = getLocationByZipCode(zip)
    df.iloc[index, df.columns.get_loc('City')] = location.major_city
    #print(f'index: {index}  Zip: {zip} --> City: {location.major_city}')

In [8]:
for row in missing_state.iterrows():
    index = row[0]
    zip = int(row[1].Zip)
    location = getLocationByZipCode(zip)
    df.iloc[index, df.columns.get_loc('State')] = location.state
    #print(f'index: {index}  Zip: {zip} --> state: {location.state}')

### count of null values in each column again

In [9]:
print(df.isnull().sum())
(df.isnull().sum() * 100 / df.index.size).round(2)

Inspection ID          0
DBA Name               0
AKA Name            2487
License #              0
Facility Type       4508
Risk                   0
Address                1
City                   0
State                  0
Zip                    0
Inspection Date        0
Inspection Type        0
Results                0
Violations         30686
Latitude               0
Longitude              0
dtype: int64


Inspection ID       0.00
DBA Name            0.00
AKA Name            1.62
License #           0.00
Facility Type       2.93
Risk                0.00
Address             0.00
City                0.00
State               0.00
Zip                 0.00
Inspection Date     0.00
Inspection Type     0.00
Results             0.00
Violations         19.97
Latitude            0.00
Longitude           0.00
dtype: float64

### Loop through violations column to find all the unique violations codes (46 total).

In [10]:
Violations_loc = df.loc[pd.notnull(df['Violations'])]
Violation_set = set()
violation_set_index = set()
for row in Violations_loc.iterrows():
    Violation_list = [x.strip() for x in row[1].Violations.split('|')]
    for item in Violation_list:
        Violation_set.add(item.split(' - Comments')[0])
        violation_set_index.add(int(item.split(' - Comments')[0].split(".")[0]))
#there are 46 unique Violations code
print(Violation_set)
# print(sorted(violation_set_index))

{'17. POTENTIALLY HAZARDOUS FOOD PROPERLY THAWED', '7. WASH AND RINSE WATER: CLEAN AND PROPER TEMPERATURE', '44. ONLY AUTHORIZED PERSONNEL IN THE FOOD-PREP AREA', '5. PERSONNEL WITH INFECTIONS RESTRICTED: NO OPEN SORES, WOUNDS, ETC', '10. SEWAGE AND WASTE WATER DISPOSAL, NO BACK SIPHONAGE, CROSS  CONNECTION AND/OR BACK FLOW', '70. NO SMOKING REGULATIONS', '18. NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS', '33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSILS CLEAN, FREE OF ABRASIVE DETERGENTS', '8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS:  CLEAN, PROPER TEMPERATURE, CONCENTRATION, EXPOSURE TIME', '34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METHODS USED', '31. CLEAN MULTI-USE UTENSILS AND SINGLE SERVICE ARTICLES PROPERLY STORED: NO REUSE OF SINGLE SERVICE ARTICLES', '23. DISHES AND UTENSILS FLUSHED, SCRAPED, SOAKED', '29. PREVIOUS MINOR VIOLATION(S

### creat new violation columns 

In [11]:
# creat new violation columns 
for x in violation_set_index:
    # print(x.split(".")[0])
    df['Violation ' + str(x)] = np.NaN

### Loop through violations column again, split and add each violations code and comments to each new corresponding column  

In [12]:
for row in Violations_loc.iterrows():
    index = row[0]
    Violation_list = [x.strip() for x in row[1].Violations.split('|')]
    for item in Violation_list:
        violation_index = item.split(' - Comments')[0].split(".")[0]
        df.iloc[index, df.columns.get_loc('Violation ' + str(violation_index))] = item

### Clean and remove the old violations column

In [13]:
df = df.drop('Violations',axis=1)
df.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Violation 37,Violation 38,Violation 39,Violation 40,Violation 41,Violation 42,Violation 43,Violation 44,Violation 45,Violation 70
0,2079132.0,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517328.0,Restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2079129.0,JET'S PIZZA,JET'S PIZZA,2522268.0,Restaurant,Risk 2 (Medium),1025 W MADISON ST,CHICAGO,IL,60607.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2079125.0,ROOM 1520,ROOM 1520,2446638.0,Special Event,Risk 3 (Low),301 N JUSTINE ST,CHICAGO,IL,60607.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2079123.0,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517338.0,Restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2079105.0,CHARTWELLS,CICS WEST BELDEN CAMPUS,2549079.0,Charter School,Risk 1 (High),2245 N MCVICKER AVE,CHICAGO,IL,60639.0,...,NaN,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...,NaN,NaN,"41. PREMISES MAINTAINED FREE OF LITTER, UNNECE...",NaN,NaN,NaN,NaN,NaN


In [14]:
df.to_csv("../Datasets/Food_Inspections_Python_cleaned.csv")